In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import scipy as sp


# read data
df = pd.read_csv('retail_price.csv')
index = ['product_category_name', 'product_id']
column_min = ['unit_price', 'product_score', 'freight_price', 'product_weight_g', 'comp_1', 'ps1', 'comp_2', 'ps2', 'comp_3', 'ps3']
# column_max = []
column_sum = ['customers']

product_category = ['bed_bath_table', 'computers_accessories', 'consoles_games', 'cool_stuff', 'furniture_decor']
                    #,
                    #'garden_tools', 'health_beauty', 'perfumery', 'watches_gifts']

table_min = pd.pivot_table(df, values=column_min, index=index,  aggfunc=np.min)
# table_min = pivot_table(df, values='D', index=index,  aggfunc=np.max)
table_sum = pd.pivot_table(df, values=column_sum, index=index, aggfunc=np.sum)

In [2]:
table = table_min.merge(table_sum, on=index,how='left')
table

comp_1      comp_2      comp_3  \
product_category_name product_id                                       
bed_bath_table        bed1         74.000000   39.240000   39.240000   
                      bed2         74.000000   74.000000   39.240000   
                      bed3         74.000000   84.900000   39.240000   
                      bed4         77.933333   44.154444   39.990000   
                      bed5         89.900000  163.398710   45.950000   
computers_accessories computers1   94.900000   94.900000   77.900000   
                      computers2  114.491154  109.900000   77.900000   
                      computers3  139.990000  134.900000   78.712281   
                      computers4  114.491154  139.990000   77.900000   
                      computers5  114.491154  119.900000   77.900000   
                      computers6  114.491154  149.900000   77.900000   
consoles_games        consoles1    19.900000   19.900000   19.900000   
                      consoles2    19.900000   19.900000   19.900000   
cool_stuff            cool1        85.704286   85.704286   29.500000   
                      cool2        89.863333  116.927500   29.500000   
                      cool3        35.000000   29.500000   29.500000   
                      cool4        89.863333  116.927500   29.500000   
                      cool5        89.863333   99.990000   29.500000   
furniture_decor       furniture1   38.400000   35.000000   38.400000   
                      furniture2   69.900000   35.000000   75.000000   
                      furniture3   38.400000   35.000000   38.400000   
                      furniture4   69.900000   35.000000   78.535714   
garden_tools          garden1      49.900000   49.900000   49.900000   
                      garden10     49.900000   49.900000   49.900000   
                      garden2      49.900000   49.900000   49.900000   
                      garden3      49.900000   49.900000   49.900000   
                      garden4      49.900000   49.900000   49.900000   
                      garden5      49.900000   49.900000   49.900000   
                      garden6      49.900000   49.900000   49.900000   
                      garden7      49.900000   49.900000   49.900000   
                      garden8      49.900000   49.900000   49.900000   
                      garden9      49.900000   49.900000   49.900000   
health_beauty         health1      23.397407   79.990000   58.990000   
                      health10     23.990000   79.990000   58.990000   
                      health2      23.397407   79.990000   58.990000   
                      health3      23.990000   79.990000   58.990000   
                      health4      23.990000   79.990000   58.990000   
                      health5      19.990000   79.990000   58.799524   
                      health6      23.990000   79.990000   58.990000   
                      health7      19.990000   64.990000   58.799524   
                      health8      19.990000   79.990000   58.799524   
                      health9      19.990000   19.990000   58.799524   
perfumery             perfumery1   49.990000   56.990000   49.990000   
                      perfumery2   49.990000  116.906667  116.906667   
watches_gifts         watches1    103.792523  105.000000  132.531250   
                      watches2    103.792523  105.000000  132.531250   
                      watches3     78.000000   78.000000  132.531250   
                      watches4    103.792523  105.000000  132.531250   
                      watches5    103.792523  105.000000  132.531250   
                      watches6    103.792523  105.000000  132.531250   
                      watches7    103.792523  105.000000  132.531250   
                      watches8    103.792523  105.000000  132.531250   

                                  freight_price  product_score  \
product_category_name product_id                                 
bed_bath_table   

## data / parameters

In [3]:
# Selling & Buying & Rating & Probability of buying
selling = {}
for category in product_category:
    selling[category] = (table.loc[category]['unit_price'].values)

buying = {}
for category in product_category:
    buying[category] = selling[category] * np.random.uniform(0.6,0.7)


rating = {}
for category in product_category:
    rating[category] = (table.loc[category]['product_score'].values)
MAX_RATING = 5.

# Customers
n_customers = {}
for category in product_category:
    n_customers[category] = (table.loc[category]['customers'].sum())

p2={}
for category in product_category:
    p2[category] = (np.exp(-selling[category]*(MAX_RATING - rating[category]) / 100) 
              / np.sum(np.exp(-selling[category]*(MAX_RATING - rating[category])/100)))


In [4]:
# fixed freight cost, can change if needed
SHIPPING_COST = 5

# how many customers we have that won't buy unless extra conditions are fufilled
MARKET_CUSTOMERS = {category: 10000 for category in product_category}

# what % of 'market' customers do we get for each % of discount?
DISCOUNT_FACTOR = 5

# what % of 'market' customers do we get if we have free shipping
FREE_SHIPPING_FACTOR = 0.1

# the budget
BUDGET = 1e9

# inventory for each category
INVENTORY_SPACE = 1e5

## model

In [5]:
m = gp.Model()

BIGNUM = 1e6 # sum(n_customers.values())

# decision variables
stock = {}
is_chosen = {}
sold = {}
discount = {}
free_shipping = {}

# initialising decision variables
for category in product_category:
    I = range(len(selling[category]))
    stock[category] = m.addVars(I,vtype = GRB.INTEGER)
    is_chosen[category] = m.addVars(I, vtype = GRB.BINARY)
    sold[category] = m.addVars(I,vtype = GRB.INTEGER)
    discount[category] = m.addVars(I,vtype=GRB.CONTINUOUS, lb=0, ub=1)
    free_shipping[category] = m.addVars(I,  vtype = GRB.BINARY)


# amount sold and extra customers
captured_customers = lambda category,i: (
    MARKET_CUSTOMERS[category] * (FREE_SHIPPING_FACTOR * free_shipping[category][i]
                                  + DISCOUNT_FACTOR * discount[category][i])
)
amount_sold  = lambda category,i: p2[category][i]* (n_customers[category] + captured_customers(category,i))

# objective function
m.setObjective(
    gp.quicksum(gp.quicksum(sold[category][i] * selling[category][i] * (1 - discount[category][i]) 
                            - stock[category][i] * buying[category][i]
                            - sold[category][i] * SHIPPING_COST * free_shipping[category][i]
                for i in range(len(selling[category]))) for category in product_category),
    GRB.MAXIMIZE
)

# Can't store past inventory space
m.addConstrs( gp.quicksum(stock[category][i] for i in range(len(selling[category]))) <= INVENTORY_SPACE 
             for category in product_category)

# Can't buy more than the budget
m.addConstr(gp.quicksum( gp.quicksum(stock[category][i] * buying[category][i] for i in range(len(selling[category]))) 
                        for category in product_category) <= BUDGET)

for category in product_category:
    # If a product is stocked, is_chosen is set to be 1 [Change to BIGNUM to remove dependencies on n_customers]
    m.addConstrs(stock[category][i] <= is_chosen[category][i] * BIGNUM
                 for i in range(len(selling[category])))
    # Can't sell more than you have stocked
    m.addConstrs(sold[category][i] <= stock[category][i] for i in range(len(selling[category])))
    # How much we can sell depends on number of customers [Removed is_chosen]
    m.addConstrs(sold[category][i] <= amount_sold(category, i)
                 for i in range(len(selling[category])))
    m.addConstrs((FREE_SHIPPING_FACTOR * free_shipping[category][i]
                                  + DISCOUNT_FACTOR * discount[category][i]) <= 1 for i in range(len(selling[category])))

m.params.NonConvex = 2
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-19
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 94 rows, 110 columns and 242 nonzeros
Model fingerprint: 0x47e471e7
Model has 44 quadratic objective terms
Variable types: 22 continuous, 88 integer (44 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+06]
  Objective range  [1e+01, 2e+02]
  QObjective range [1e+01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+09]
Found heuristic solution: objective -0.0000000
Presolve removed 92 rows and 107 columns
Presolve time: 0.54s
Presolved: 6 rows, 6 columns, 15 nonzeros
Presolved model has 1 bilinear constraint(s)
Found heuristic solution: objective 602399.82232
Variable types: 3 c

In [6]:
discount

{'bed_bath_table': {0: <gurobi.Var C15 (value 0.14663124789533197)>,
  1: <gurobi.Var C16 (value 0.1466174898543905)>,
  2: <gurobi.Var C17 (value 0.14673724578978678)>,
  3: <gurobi.Var C18 (value 0.14662660639523803)>,
  4: <gurobi.Var C19 (value 0.14664620746804222)>},
 'computers_accessories': {0: <gurobi.Var C43 (value 0.10300187129617129)>,
  1: <gurobi.Var C44 (value 0.10295396549417402)>,
  2: <gurobi.Var C45 (value 0.10297678443573077)>,
  3: <gurobi.Var C46 (value 0.10299133938755596)>,
  4: <gurobi.Var C47 (value 0.10301756216573445)>,
  5: <gurobi.Var C48 (value 0.10292743458270379)>},
 'consoles_games': {0: <gurobi.Var C61 (value 0.16973296992174655)>,
  1: <gurobi.Var C62 (value 0.16974746255491227)>},
 'cool_stuff': {0: <gurobi.Var C80 (value 0.1421996768330323)>,
  1: <gurobi.Var C81 (value 0.14226145936215273)>,
  2: <gurobi.Var C82 (value 0.14224388472630212)>,
  3: <gurobi.Var C83 (value 0.14223360259951012)>,
  4: <gurobi.Var C84 (value 0.14225602824301964)>},
 'fur

In [7]:
free_shipping

{'bed_bath_table': {0: <gurobi.Var C20 (value 0.0)>,
  1: <gurobi.Var C21 (value 0.0)>,
  2: <gurobi.Var C22 (value 0.0)>,
  3: <gurobi.Var C23 (value 0.0)>,
  4: <gurobi.Var C24 (value -0.0)>},
 'computers_accessories': {0: <gurobi.Var C49 (value 0.0)>,
  1: <gurobi.Var C50 (value 0.0)>,
  2: <gurobi.Var C51 (value 0.0)>,
  3: <gurobi.Var C52 (value -0.0)>,
  4: <gurobi.Var C53 (value 0.0)>,
  5: <gurobi.Var C54 (value -0.0)>},
 'consoles_games': {0: <gurobi.Var C63 (value 0.0)>,
  1: <gurobi.Var C64 (value 0.0)>},
 'cool_stuff': {0: <gurobi.Var C85 (value 0.0)>,
  1: <gurobi.Var C86 (value 0.0)>,
  2: <gurobi.Var C87 (value 0.0)>,
  3: <gurobi.Var C88 (value 0.0)>,
  4: <gurobi.Var C89 (value 0.0)>},
 'furniture_decor': {0: <gurobi.Var C106 (value 0.0)>,
  1: <gurobi.Var C107 (value -0.0)>,
  2: <gurobi.Var C108 (value 0.0)>,
  3: <gurobi.Var C109 (value -0.0)>}}

In [8]:
stock

{'bed_bath_table': {0: <gurobi.Var C0 (value 2956.0)>,
  1: <gurobi.Var C1 (value 1939.0)>,
  2: <gurobi.Var C2 (value 1034.0)>,
  3: <gurobi.Var C3 (value 3074.0)>,
  4: <gurobi.Var C4 (value 1642.0)>},
 'computers_accessories': {0: <gurobi.Var C25 (value 2294.0)>,
  1: <gurobi.Var C26 (value 1384.0)>,
  2: <gurobi.Var C27 (value 1537.0)>,
  3: <gurobi.Var C28 (value 1782.0)>,
  4: <gurobi.Var C29 (value 1400.0)>,
  5: <gurobi.Var C30 (value 1559.0)>},
 'consoles_games': {0: <gurobi.Var C55 (value 4596.0)>,
  1: <gurobi.Var C56 (value 4330.0)>},
 'cool_stuff': {0: <gurobi.Var C65 (value 1845.0)>,
  1: <gurobi.Var C66 (value 1616.0)>,
  2: <gurobi.Var C67 (value 3069.0)>,
  3: <gurobi.Var C68 (value 725.0)>,
  4: <gurobi.Var C69 (value 2025.0)>},
 'furniture_decor': {0: <gurobi.Var C90 (value 2795.0)>,
  1: <gurobi.Var C91 (value 2349.0)>,
  2: <gurobi.Var C92 (value 2909.0)>,
  3: <gurobi.Var C93 (value 2755.0)>}}